In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import re
import pandas as pd

In [2]:
def get_dirid(url):
    match = re.search(r'dirId=12(\d+)', url)
    dirid_value = match.group(1)
    return dirid_value

In [3]:
def get_docid(url):
    match = re.search(r'docId=(\d+)', url)
    dirid_value = match.group(1)
    return dirid_value

In [4]:
# dirID
def get_loc(id=""):
    url = 'https://kin.naver.com/qna/list.naver?dirId=12'+id

    # 페이지 요청
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    
    # BeautifulSoup으로 페이지 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 질문 제목 추출
    location = soup.find(attrs={'class': 'location'}).get_text(strip=True)
    return location

In [5]:
def find_pattern(text):
    # 문자열 길이의 절반까지 탐색(더 긴 패턴은 두 번 반복될 수 없으므로)
    for length in range(1, len(text)):
        # 길이 `length`만큼 패턴을 잘라서 비교
        pattern = text[:length]
        # 패턴이 두 번 연속 반복되는지 확인
        if text.startswith(pattern * 2):
            return pattern

In [6]:
# 지역 & 플레이스 기본 정보 문자열 제거를 위한 변수 지정
url2 = 'https://kin.naver.com/qna/list.naver?dirId=12'
headers2 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response2 = requests.get(url2, headers=headers2)
soup2 = BeautifulSoup(response2.text, 'html.parser')
useless_information = soup2.find(attrs={'class': 'location'}).get_text(strip=True)

def return_location(search):
    a = get_loc(search[:-2]+"01").replace(get_loc(search[:-2]),"")
    a_elim = get_loc(search[:-len(search)]+search[-len(search):-2]).replace(get_loc(search[:-len(search)]),"")
    first_loc = find_pattern(a)
    a = a[len(first_loc):]
    b = get_loc(search).replace(a,"")
    c = b.replace(useless_information, "")
    return c.replace(a_elim,"")
    
def return_full_location(search):
    r = ""
    n = 2
    while n <= len(search):
        r += return_location(search[:n])+ " "
        n += 2
    return r

In [7]:
def show_information(url):
    # 페이지 요청
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    
    # BeautifulSoup으로 페이지 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 질문 제목 추출
    title = soup.find(attrs={'class': 'endTitleSection'}).get_text(strip=True)
    Q_detail= soup.find(attrs={'class': 'questionDetail'}).get_text(strip=True,separator='\n')
    try:
        answer = soup.find(attrs={'class': 'se-module se-module-text'}).get_text(strip = True, separator = "\n")
    except:
        answer = "아직 답변이 없습니다"
    
    print("제목:\n", title, "\n", '-'*40)
    print("질문 내용:\n", Q_detail,'\n','-'*40)
    print("답변:\n", answer,'\n','-'*40)
    print("위치:\n", return_full_location(get_dirid(url)), '\n', '-'*40)
    print("문서 번호:\n", get_docid(url), '\n', '-'*40)

In [8]:
url = 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12011701&docId=475395637'

show_information(url)

제목:
 질문미아사거리 놀 곳 어디 있는지 알려주세요 
 ----------------------------------------
질문 내용:
 곳 가는데 갈 곳이 없어서요ㅜ 부탁드려요 
 ----------------------------------------
답변:
 미아사거리역   주변 추천지 입니다
​
1.장소 입니다
보드게임카페
만화카페
디저트카페
볼링장
포켓볼
탁구장
인생네컷
인형뽑기방
영화관
PC방
맛집가기
서점가기
​
2.명소 입니다
북서울꿈의 숲
현대백화점
롯데백화점 
 ----------------------------------------
위치:
 서울특별시 강북구 미아동  
 ----------------------------------------
문서 번호:
 475395637 
 ----------------------------------------


In [9]:
url2 = 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1209150111&docId=475397693'

show_information(url2)

제목:
 질문분당고 근처 스카나 독서실 추천 
 ----------------------------------------
질문 내용:
 추석 때 분당고 근처로 가서,,, 하루 정도 공부할 수 있는 경기도에 있는 분당고 근처 스카나 독서실 좋은 곳 추천해주실 수 있나욤,, 
 ----------------------------------------
답변:
 아직 답변이 없습니다 
 ----------------------------------------
위치:
 경기도 성남시 분당구 수내동  
 ----------------------------------------
문서 번호:
 475397693 
 ----------------------------------------


In [10]:
url3 = 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1209160305&docId=20450042&qb=7ISx6reg6rSA64yAIOyVng==&enc=utf8&section=kin.qna_ency&rank=1&search_sort=0&spq=0'

show_information(url3)

제목:
 질문수원 성균관대 앞에서 112번이나 118번 버스 노선 잘 아시는분 
 ----------------------------------------
질문 내용:
 성균관대 앞에서 112번이나 118번을 타면 권선2지구 간다고 들었는데요.
제가 과외 때문에 권선주공2차 아파트 를 가야 할것 같은데,,
수원쪽 교통은 전혀 몰라서요,,
알아내고 알아낸게 저 버스 두개 밖에 없는데요,,
성균관대에서 권선동 주공2차 아파트 까지 갈수 있는 가장 가까운 버스 노선 아시는분 있으면 좀 알려주시겠어요..
시간이 어느정도 걸리는지도요.
답변해주시면 정말 감사드릴께요,,^^ 
 ----------------------------------------
답변:
 아직 답변이 없습니다 
 ----------------------------------------
위치:
 경기도 수원시 장안구 율전동  
 ----------------------------------------
문서 번호:
 20450042 
 ----------------------------------------


In [11]:
url4 = 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12013723&docId=20855086&qb=7ISx6reg6rSA64yAIOyVng==&enc=utf8&section=kin.qna_ency&rank=7&search_sort=0&spq=0'

show_information(url4)

제목:
 질문성균관대 앞 빙수킹 아직도 있나요? 
 ----------------------------------------
질문 내용:
 성균관대 앞 빙수킹 아직도 있나요?
정확한 위치 좀 알려주세요
전에 한번 찾아갔는데 못 찾겠더라구요.
맛있나요? 
 ----------------------------------------
답변:
 아직 답변이 없습니다 
 ----------------------------------------
위치:
 서울특별시 종로구 명륜동  
 ----------------------------------------
문서 번호:
 20855086 
 ----------------------------------------


In [12]:
url5 = 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1209050102&docId=475351028'

show_information(url5)

제목:
 질문고양종합운동장 근처 포토이즘 
 ----------------------------------------
질문 내용:
 고양종합운동장 근처에 포토이즘 있나요?
있다면 지점명 알려주세요!
콘서트 가는김에 응원봉이나 굿즈 들고 찍고싶어요 
 ----------------------------------------
답변:
 고양종합운동장, 대화역, 킨텍스 쪽엔 없습니다
그나마 제일 가까운 곳이라면
정발산역 라페스타, 웨스턴돔에 있습니다 
 ----------------------------------------
위치:
 경기도 고양시 덕양구 고양동  
 ----------------------------------------
문서 번호:
 475351028 
 ----------------------------------------
